<a href="https://colab.research.google.com/github/Shtepser/Manyinterpret/blob/master/Day2/%D0%A2%D1%8E%D0%BC%D0%B5%D0%BD%D1%8C_BERT_%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_30042022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jsonlines
!pip install transformers

     |████████████████████████████████| 4.2 MB 31.5 MB/s 
     |████████████████████████████████| 596 kB 59.9 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 6.6 MB 53.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Классификация предложений

## Загрузка данных

In [ ]:
!wget https://dl.fbaipublicfiles.com/glue/superglue/data/v2/RTE.zip
!unzip RTE.zip

--2022-05-07 06:14:34--  https://dl.fbaipublicfiles.com/glue/superglue/data/v2/RTE.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 750920 (733K) [application/zip]
Saving to: ‘RTE.zip’

RTE.zip             100%[===================>] 733.32K  1.93MB/s    in 0.4s    

2022-05-07 06:14:35 (1.93 MB/s) - ‘RTE.zip’ saved [750920/750920]

Archive:  RTE.zip
   creating: RTE/
  inflating: RTE/train.jsonl         
  inflating: RTE/test.jsonl          
  inflating: RTE/val.jsonl           


In [ ]:
import jsonlines

with jsonlines.open("RTE/train.jsonl", "r") as fin:
    train_data = list(fin)
with jsonlines.open("RTE/val.jsonl", "r") as fin:
    dev_data = list(fin)
with jsonlines.open("RTE/test.jsonl", "r") as fin:
    test_data = list(fin)
  
train_data[:10]

[{'hypothesis': 'Weapons of Mass Destruction Found in Iraq.',
  'idx': 0,
  'label': 'not_entailment',
  'premise': 'No Weapons of Mass Destruction Found in Iraq Yet.'},
 {'hypothesis': 'Pope Benedict XVI is the new leader of the Roman Catholic Church.',
  'idx': 1,
  'label': 'entailment',
  'premise': 'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.'},
 {'hypothesis': 'Herceptin can be used to treat breast cancer.',
  'idx': 2,
  'label': 'entailment',
  'premise': 'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.'},
 {'hypothesis': 'The previous name of Ho Chi Minh City was Saigon.',
  'idx': 3,
  'label': 'entailment',
  'premise': 'Judie Vivian, chief executive at ProMedica, a 

токенизатор преобразует слова в индексы

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
for key, value in train_data[9].items():
    print(key, value)
tokenized_data = tokenizer(train_data[9]["hypothesis"])
print(tokenized_data)
token_indexes = tokenized_data["input_ids"]
tokens = tokenizer.convert_ids_to_tokens(token_indexes)
print("")
for index, token in zip(token_indexes, tokens):
    print(index, token)

premise Only a week after it had no comment on upping the storage capacity of its Hotmail e-mail service, Microsoft early Thursday announced it was boosting the allowance to 250MB to follow similar moves by rivals such as Google, Yahoo, and Lycos.
hypothesis Microsoft's Hotmail has raised its storage capacity to 250MB.
label entailment
idx 9
{'input_ids': [101, 6998, 112, 188, 4126, 14746, 1144, 2120, 1157, 5092, 3211, 1106, 4805, 20660, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

101 [CLS]
6998 Microsoft
112 '
188 s
4126 Hot
14746 ##mail
1144 has
2120 raised
1157 its
5092 storage
3211 capacity
1106 to
4805 250
20660 ##MB
119 .
102 [SEP]


токенизатор можно применять и к парам предложений

In [ ]:
tokenized_data = tokenizer(train_data[9]["premise"], train_data[9]["hypothesis"])
print(tokenized_data["token_type_ids"])
token_indexes = tokenized_data["input_ids"]
tokens = tokenizer.convert_ids_to_tokens(token_indexes)
print("")
for index, token in zip(token_indexes, tokens):
    print(index, token)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

101 [CLS]
2809 Only
170 a
1989 week
1170 after
1122 it
1125 had
1185 no
7368 comment
1113 on
1146 up
2624 ##ping
1103 the
5092 storage
3211 capacity
1104 of
1157 its
4126 Hot
14746 ##mail
174 e
118 -
6346 mail
1555 service
117 ,
6998 Microsoft
1346 early
9170 Thursday
1717 announced
1122 it
1108 was
14112 boost
1158 ##ing
1103 the
25916 allowance
1106 to
4805 250
20660 ##MB
1106 to
2812 follow
1861 similar
5279 moves
1118 by
9521 rivals
1216 such
1112 as
7986 Google
117 ,
23535 Yahoo
117 ,
1105 and
149 L
1183 ##y
13538 ##cos
119 .
102 [SEP]
6998 Microsoft
112 '
188 s
4126 Hot
14746 ##mail
1144 has
2120 raised
1157 its
5092 storage
3211 capacity
1106 to
4805 250
20660 ##MB
119 .
102 [SEP]


напишем соответствующий `Dataset`

In [ ]:
from torch.utils.data.dataset import Dataset

class RTEDataset(Dataset):

    def __init__(self, data, tokenizer, pos_label="entailment"):
        self.data = data
        self.tokenizer = tokenizer
        self.pos_label = pos_label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        answer = self.tokenizer(item["premise"], item["hypothesis"])
        if "label" in item:
            answer["y"] = float(item["label"] == self.pos_label)
        return answer 

In [ ]:
train_dataset = RTEDataset(train_data, tokenizer)
for key, value in train_dataset[10].items():
    print(key, value, sep="\n")

input_ids
[101, 12221, 1161, 10768, 117, 3565, 117, 1108, 1255, 138, 1584, 10655, 19183, 21690, 1106, 12808, 2153, 117, 1105, 1108, 2120, 1112, 170, 4360, 119, 5355, 112, 188, 7119, 24512, 4438, 1104, 4483, 117, 1133, 1118, 1644, 117, 1155, 5237, 12808, 1116, 1132, 4360, 119, 10768, 4213, 1106, 7522, 1120, 1425, 1744, 117, 1105, 1170, 1199, 18561, 21611, 7866, 1125, 1123, 1417, 10368, 2014, 1107, 1729, 119, 1438, 117, 1113, 1123, 1422, 2428, 3556, 1569, 10999, 117, 1103, 1305, 25510, 18794, 1951, 5366, 1123, 2202, 4483, 1112, 6489, 119, 1130, 1546, 1106, 1138, 1123, 4483, 2014, 117, 1103, 1305, 25510, 18794, 1951, 1163, 10768, 1156, 1138, 1106, 6268, 170, 10890, 1104, 170, 21341, 2225, 1183, 1121, 1103, 4360, 156, 16234, 1161, 2031, 119, 102, 12221, 1161, 10768, 112, 188, 2153, 1132, 1121, 5355, 119, 102]
token_type_ids
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
import torch
import numpy as np
import itertools

def pad_tensor(vec, length, dim, pad_symbol):
    # vec.shape = [3, 4, 5]
    # length=7, dim=1 -> pad_size = (3, 7-4, 5)
    pad_size = list(vec.shape)
    pad_size[dim] = length - vec.shape[dim]
    answer = torch.cat([vec, torch.ones(*pad_size, dtype=torch.long) * pad_symbol], axis=dim)
    return answer

def pad_tensors(tensors, pad=0):
    # дополняет тензоры из tensors до общей максимальной длины символом pad
    if isinstance(tensors[0], (int, np.integer)):
        return torch.LongTensor(tensors)
    elif isinstance(tensors[0], (float, np.float)):
        return torch.Tensor(tensors)
    tensors = [torch.LongTensor(tensor) for tensor in tensors]
    L = max(tensor.shape[0] for tensor in tensors)
    tensors = [pad_tensor(tensor, L, dim=0, pad_symbol=pad) for tensor in tensors]
    return torch.stack(tensors, axis=0)

class FieldBatchDataLoader:

    def __init__(self, X, batch_size=32, sort_by_length=True, 
                 length_field=None, state=115, device="cpu"):
        self.X = X
        self.batch_size = batch_size
        self.sort_by_length = sort_by_length
        self.length_field = length_field  ## добавилось
        self.device = device
        np.random.seed(state)

    def __len__(self):
        return (len(self.X)-1) // self.batch_size + 1 

    def __iter__(self):
        if self.sort_by_length:
            # отсортировать индексы по длине объектов [1, ..., 32] -> [7, 4, 15, ...]
            # изменилось взятие длины из поля
            if self.length_field is not None:
                lengths = [len(x[self.length_field]) for x in self.X]
            else:
                lengths = [len(list(x.values())[0]) for x in self.X]
            order = np.argsort(lengths)
            # сгруппировать в батчи [7, 4, 15, 31, 3, ...] -> [[7, 4, 15, 31], [3, ...], ...]
            batched_order = np.array([order[start:start+self.batch_size] 
                                      for start in range(0, len(self.X), self.batch_size)])
            # переупорядочить батчи случайно: [[3, 11, 21, 19], [27, ...], ..., [7, ...], ...]
            np.random.shuffle(batched_order[:-1])
            # собрать посл-ть индексов: -> [3, 11, 21, 19, 27, ...]
            self.order = np.fromiter(itertools.chain.from_iterable(batched_order), dtype=int)
        else:
            self.order = np.arange(len(self.X))
            np.random.shuffle(self.order)
        self.idx = 0
        return self

    def __next__(self):
        if self.idx >= len(self.X):
            raise StopIteration()
        end = min(self.idx + self.batch_size, len(self.X))
        indexes = [self.order[i] for i in range(self.idx, end)]
        batch = dict()
        # перебираем все поля
        for field in self.X[indexes[0]]:
            batch[field] = pad_tensors([self.X[i][field] for i in indexes]).to(self.device)
        batch["indexes"] = indexes
        self.idx = end
        return batch

In [ ]:
train_dataloader = iter(FieldBatchDataLoader(train_dataset, batch_size=8, device="cuda"))
for i in range(10):
    batch = next(train_dataloader)
    for field, data in batch.items():
        print(f"{field}:{np.shape(data)}", end="\t")
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


input_ids:torch.Size([8, 48])	token_type_ids:torch.Size([8, 48])	attention_mask:torch.Size([8, 48])	y:torch.Size([8])	indexes:(8,)	
input_ids:torch.Size([8, 189])	token_type_ids:torch.Size([8, 189])	attention_mask:torch.Size([8, 189])	y:torch.Size([8])	indexes:(8,)	
input_ids:torch.Size([8, 71])	token_type_ids:torch.Size([8, 71])	attention_mask:torch.Size([8, 71])	y:torch.Size([8])	indexes:(8,)	
input_ids:torch.Size([8, 49])	token_type_ids:torch.Size([8, 49])	attention_mask:torch.Size([8, 49])	y:torch.Size([8])	indexes:(8,)	
input_ids:torch.Size([8, 181])	token_type_ids:torch.Size([8, 181])	attention_mask:torch.Size([8, 181])	y:torch.Size([8])	indexes:(8,)	
input_ids:torch.Size([8, 27])	token_type_ids:torch.Size([8, 27])	attention_mask:torch.Size([8, 27])	y:torch.Size([8])	indexes:(8,)	
input_ids:torch.Size([8, 262])	token_type_ids:torch.Size([8, 262])	attention_mask:torch.Size([8, 262])	y:torch.Size([8])	indexes:(8,)	
input_ids:torch.Size([8, 141])	token_type_ids:torch.Size([8, 141])	

## Создание и обучение модели

In [ ]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained("bert-base-cased").to("cuda")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_output = bert_model(batch['input_ids'], batch['token_type_ids'])
print(type(bert_output))
for key, value in bert_output.items():
    print(key, value.shape)

<class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>
last_hidden_state torch.Size([8, 49, 768])
pooler_output torch.Size([8, 768])


напишем класс-обёртку для классификации

In [ ]:
import torch.nn as nn
from transformers.optimization import AdamW

class BasicTransformersClassificationModel(nn.Module):

    def __init__(self, model, labels_number, lr=1e-5, device="cpu", **kwargs):
        super(BasicTransformersClassificationModel, self).__init__()
        self.model = model
        self.labels_number = labels_number
        self.build_network(labels_number)
        # определяем функцию потерь
        if self.labels_number > 1:
            self.log_softmax = nn.LogSoftmax(dim=-1)
            self.criterion = nn.NLLLoss(reduction="mean")
        else:
            self.log_softmax = nn.Sigmoid() # sigma(x) = 1/ (1+exp(-x))
            self.criterion = nn.BCELoss(reduction="mean")
        self.device = device
        if self.device is not None:
            self.to(self.device)
        self.optimizer = AdamW(self.parameters(), lr=lr, weight_decay=0.01)

    @property
    def hidden_size(self):
        return self.model.config.hidden_size

    def forward(self, input_ids, **kwargs):
        raise NotImplementedError("You should implement forward pass in your derived class.") 

    def train_on_batch(self, x, y):
        self.train()
        self.optimizer.zero_grad()
        loss = self._validate(x, y)
        loss["loss"].backward()
        self.optimizer.step()
        return loss

    def validate_on_batch(self, x, y):
        self.eval()
        with torch.no_grad():
            return self._validate(x, y)

    def _validate(self, x, y):
        if self.device is not None:
            y = y.to(self.device)
        log_probs = self(**x) #   self.forward(x) = self.__call__(x)
        loss = self.criterion(log_probs, y)
        if self.labels_number > 1:
            _, labels = torch.max(log_probs, dim=-1)
        else:
            labels = (log_probs >= 0.5).int()
        return {"loss": loss, "labels": labels}

class TransformersClassificationModel(BasicTransformersClassificationModel):

    def build_network(self, labels_number):
        self.proj_layer = torch.nn.Linear(self.hidden_size, self.labels_number)
        return self

    def forward(self, input_ids, token_type_ids=None, **kwargs):
        input_ids = input_ids.to(self.device)
        if token_type_ids is not None:
            token_type_ids = token_type_ids.to(self.device)
        cls_output = self.model(input_ids, token_type_ids)["pooler_output"]
        logits = self.proj_layer(cls_output)
        log_probs = self.log_softmax(logits)
        if self.labels_number == 1:
            log_probs = log_probs[...,0]
        return log_probs


In [ ]:
model = TransformersClassificationModel(
    bert_model, labels_number=1, device="cuda"
)
for name, elem in model.named_parameters():
    print(name, elem.device, elem.shape)

model.embeddings.word_embeddings.weight cuda:0 torch.Size([28996, 768])
model.embeddings.position_embeddings.weight cuda:0 torch.Size([512, 768])
model.embeddings.token_type_embeddings.weight cuda:0 torch.Size([2, 768])
model.embeddings.LayerNorm.weight cuda:0 torch.Size([768])
model.embeddings.LayerNorm.bias cuda:0 torch.Size([768])
model.encoder.layer.0.attention.self.query.weight cuda:0 torch.Size([768, 768])
model.encoder.layer.0.attention.self.query.bias cuda:0 torch.Size([768])
model.encoder.layer.0.attention.self.key.weight cuda:0 torch.Size([768, 768])
model.encoder.layer.0.attention.self.key.bias cuda:0 torch.Size([768])
model.encoder.layer.0.attention.self.value.weight cuda:0 torch.Size([768, 768])
model.encoder.layer.0.attention.self.value.bias cuda:0 torch.Size([768])
model.encoder.layer.0.attention.output.dense.weight cuda:0 torch.Size([768, 768])
model.encoder.layer.0.attention.output.dense.bias cuda:0 torch.Size([768])
model.encoder.layer.0.attention.output.LayerNorm.wei

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
train_dataloader = iter(FieldBatchDataLoader(train_dataset, batch_size=8, device="cuda"))
batch = next(train_dataloader)
labels = batch["y"]
print(labels)
for i in range(100):
    loss = model.train_on_batch(batch, labels)["loss"].item()
    if i < 5 or (i+1) % 10 == 0:
        print(i, loss)
print(model.validate_on_batch(batch, labels)["loss"].item())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


tensor([1., 1., 1., 0., 0., 0., 0., 1.], device='cuda:0')
0 0.7128400206565857
1 0.7018702030181885
2 0.6858839392662048
3 0.6689372062683105
4 0.6382862329483032
9 0.5524446964263916
19 0.3595888614654541
29 0.2368309199810028
39 0.12382259964942932
49 0.0721612200140953
59 0.039024632424116135
69 0.02782338857650757
79 0.020753618329763412
89 0.01589852385222912
99 0.011956075206398964
0.007048653904348612


In [ ]:
!nvidia-smi

Sat May  7 06:40:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    35W /  70W |  15090MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Обучение классификационной модели

In [ ]:
def update_metrics(metrics, batch_output, batch_labels):
    # код только для бинарных меток!!!!
    n_batches = metrics["n_batches"]
    metrics["loss"] = (metrics["loss"] * n_batches + batch_output["loss"].item()) / (n_batches + 1)
    metrics["n_batches"] += 1
    are_equal = (batch_output["labels"] == batch_labels).float()
    are_TP = are_equal * batch_labels
    are_FP = (1.0 - are_equal) * (1-batch_labels)
    are_FN = (1.0 - are_equal) * batch_labels
    metrics["correct"] += are_equal.sum().item()
    metrics["total"] += are_equal.shape[0]
    metrics["TP"] += are_TP.sum().item()
    metrics["FP"] += are_FP.sum().item()
    metrics["FN"] += are_FN.sum().item()
    metrics["accuracy"] = metrics["correct"] / max(metrics["total"], 1)
    metrics["P"] = metrics["TP"] / max(metrics["TP"]+metrics["FP"], 1)
    metrics["R"] = metrics["TP"] / max(metrics["TP"]+metrics["FN"], 1)
    metrics["F1"] = metrics["TP"] / max(metrics["TP"]+0.5*metrics["FN"]+0.5*metrics["FP"], 1)

In [ ]:
import tqdm

def do_epoch(model, dataloader, mode="validate", epoch=1):
    metrics = {"correct": 0, "total": 0, "loss": 0.0, "n_batches": 0,
               "TP": 0, "FP": 0, "FN": 0}
    func = model.train_on_batch if mode == "train" else model.validate_on_batch
    progress_bar = tqdm.notebook.tqdm(dataloader, leave=True)
    progress_bar.set_description(f"{mode}, epoch={epoch}")
    for batch in progress_bar:
        batch_answers = batch["y"]
        batch_output = func(batch, batch_answers)
        update_metrics(metrics, batch_output, batch_answers)
        postfix = {"loss": round(metrics["loss"], 4), "acc": round(100 * metrics["accuracy"], 2)}
        for key in ["P", "R", "F1"]:
            postfix[key] =  round(100 * metrics[key], 2)
        progress_bar.set_postfix(postfix)
    return metrics

In [ ]:
train_dataset = RTEDataset(train_data, tokenizer)
dev_dataset = RTEDataset(dev_data, tokenizer)
train_dataloader = iter(FieldBatchDataLoader(train_dataset, batch_size=8, device="cuda"))
dev_dataloader = iter(FieldBatchDataLoader(dev_dataset, batch_size=8, device="cuda"))

bert_model = AutoModel.from_pretrained("bert-base-cased").to("cuda")
model = TransformersClassificationModel(bert_model, labels_number=1, device="cuda")
best_val_acc = 0.0
checkpoint = "checkpoint_best.pt"
for epoch in range(10):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)
        # print("Saving ")
model.load_state_dict(torch.load(checkpoint))
do_epoch(model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Bert

  0%|          | 0/312 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

{'F1': 0.62,
 'FN': 53.0,
 'FP': 61.0,
 'P': 0.6038961038961039,
 'R': 0.636986301369863,
 'TP': 93.0,
 'accuracy': 0.5884476534296029,
 'correct': 163.0,
 'loss': 1.2453368604183197,
 'n_batches': 35,
 'total': 277}

In [ ]:
def predict_with_model(model, X, classes):
    model.eval()
    dataloader = FieldBatchDataLoader(X, device=model.device)
    answer = [None] * len(X)
    for batch in dataloader:
        with torch.no_grad():
            batch_answer = model(**batch)
        # только для бинарной классификации
        labels = (batch_answer >= 0.5).int().cpu().numpy()
        for i, prob, label in zip(batch["indexes"], batch_answer, labels):
            # print("PROB", prob)
            answer[i] = {'label': classes[label], 'prob': prob.item()}
    return answer

In [ ]:
predicted = predict_with_model(model, dev_dataset, classes=["not_entailment", "entailment"])
for i in range(10,20):
    print(dev_data[i]["premise"])
    print(dev_data[i]["hypothesis"])
    print(dev_data[i]["label"], predicted[i]['label'], f"{100*predicted[i]['prob']:.2f}")
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


The international humanitarian aid organization, Doctors Without Borders/Medecins Sans Frontieres (MSF), continues to treat victims of violence in all locations where it is present in Darfur.
Doctors Without Borders is an international aid organization.
entailment not_entailment 45.27

In a bowl, whisk together the eggs and sugar until completely blended and frothy.
In a bowl, whisk together the egg, sugar and vanilla until light in color.
not_entailment entailment 94.89

In Nigeria, by far the most populous country in sub-Saharan Africa, over 2.7 million people are infected with HIV.
2.7 percent of the people infected with HIV live in Africa.
not_entailment not_entailment 0.68

A jury is slated to decide for the first time whether Jack Kevorkian, famed as "Dr. Death," has violated Michigan's assisted-suicide ban, while the state continues to grapple with the issue of what to allow when the ill want to end their pain by ending their lives.
Jack Kevorkian is the real name of "Dr. Death"

In [ ]:
!head -n 10 RTE/test.jsonl

{"premise": "Mangla was summoned after Madhumita's sister Nidhi Shukla, who was the first witness in the case.", "hypothesis": "Shukla is related to Mangla.", "idx": 0}
{"premise": "Authorities in Brazil say that more than 200 people are being held hostage in a prison in the country's remote, Amazonian-jungle state of Rondonia.", "hypothesis": "Authorities in Brazil hold 200 people as hostage.", "idx": 1}
{"premise": "A mercenary group faithful to the warmongering policy of former Somozist colonel Enrique Bermudez attacked an IFA truck belonging to the interior ministry at 0900 on 26 March in El Jicote, wounded and killed an interior ministry worker and wounded five others.", "hypothesis": "An interior ministry worker was killed by a mercenary group.", "idx": 2}
{"premise": "The British ambassador to Egypt, Derek Plumbly, told Reuters on Monday that authorities had compiled the list of 10 based on lists from tour companies and from families whose relatives have not been in contact sinc

### Замена BERT на Roberta

In [ ]:
from transformers import AutoTokenizer
MODEL_NAME = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = RTEDataset(train_data, tokenizer)
dev_dataset = RTEDataset(dev_data, tokenizer)
train_dataloader = iter(FieldBatchDataLoader(train_dataset, batch_size=8, device="cuda"))
dev_dataloader = iter(FieldBatchDataLoader(dev_dataset, batch_size=8, device="cuda"))

bert_model = AutoModel.from_pretrained(MODEL_NAME).to("cuda")
model = TransformersClassificationModel(bert_model, labels_number=1, device="cuda")
best_val_acc = 0.0
checkpoint = "checkpoint_best.pt"
for epoch in range(5):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)
        # print("Saving ")
model.load_state_dict(torch.load(checkpoint))
do_epoch(model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Bert

  0%|          | 0/312 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

{'F1': 0.7947019867549668,
 'FN': 26.0,
 'FP': 36.0,
 'P': 0.7692307692307693,
 'R': 0.821917808219178,
 'TP': 120.0,
 'accuracy': 0.776173285198556,
 'correct': 215.0,
 'loss': 0.6822968974709511,
 'n_batches': 35,
 'total': 277}

In [ ]:
predicted = predict_with_model(model, dev_dataset, classes=["not_entailment", "entailment"])
for i in range(10,20):
    print(dev_data[i]["premise"])
    print(dev_data[i]["hypothesis"])
    print(dev_data[i]["label"], predicted[i]['label'], f"{100*predicted[i]['prob']:.2f}")
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


The international humanitarian aid organization, Doctors Without Borders/Medecins Sans Frontieres (MSF), continues to treat victims of violence in all locations where it is present in Darfur.
Doctors Without Borders is an international aid organization.
entailment entailment 99.04

In a bowl, whisk together the eggs and sugar until completely blended and frothy.
In a bowl, whisk together the egg, sugar and vanilla until light in color.
not_entailment not_entailment 25.24

In Nigeria, by far the most populous country in sub-Saharan Africa, over 2.7 million people are infected with HIV.
2.7 percent of the people infected with HIV live in Africa.
not_entailment not_entailment 7.57

A jury is slated to decide for the first time whether Jack Kevorkian, famed as "Dr. Death," has violated Michigan's assisted-suicide ban, while the state continues to grapple with the issue of what to allow when the ill want to end their pain by ending their lives.
Jack Kevorkian is the real name of "Dr. Death"

In [ ]:
data_sample = {
    "premise": "Teenager Carlos Alcaraz prevailed over Rafa Nadal in a Spanish clash of the generations at the Madrid Open on Friday, claiming his first victory over his idol.",
    "hypothesis": "Alcaraz has beaten Nadal in Madrid Open."
}
test_dataset = RTEDataset([data_sample], tokenizer)
predictions = predict_with_model(model, test_dataset, classes=["not_entailment", "entailment"])
print(predictions[0])

{'label': 'entailment', 'prob': 0.6727221012115479}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


## Классификация последовательностей

In [ ]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Spanish-AnCora/master/es_ancora-ud-train.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Spanish-AnCora/master/es_ancora-ud-dev.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Spanish-AnCora/master/es_ancora-ud-test.conllu

--2022-05-14 06:05:33--  https://raw.githubusercontent.com/UniversalDependencies/UD_Spanish-AnCora/master/es_ancora-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44032969 (42M) [text/plain]
Saving to: ‘es_ancora-ud-train.conllu’

es_ancora-ud-train. 100%[===================>]  41.99M   159MB/s    in 0.3s    

2022-05-14 06:05:36 (159 MB/s) - ‘es_ancora-ud-train.conllu’ saved [44032969/44032969]

--2022-05-14 06:05:36--  https://raw.githubusercontent.com/UniversalDependencies/UD_Spanish-AnCora/master/es_ancora-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connect

In [ ]:
!head -n 70 es_ancora-ud-test.conllu

# newdoc id = 3LB-CAST-111_C-4
# sent_id = 3LB-CAST-111_C-4-s1
# text = Partidario de la "perestroika" de Mijail Gorbachov en la Unión Soviética, en 1989 entró en conflicto con Yívkov, líder durante 35 años del Partido Comunista y del Estado búlgaro, y le acusó en una carta abierta de utilizar métodos poco democráticos de gobierno.
# orig_file_sentence 001#1
1	Partidario	partidario	ADJ	aq0ms0	Gender=Masc|Number=Sing	17	amod	17:amod	_
2	de	de	ADP	sps00	_	5	case	5:case	_
3	la	el	DET	da0fs0	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	5	det	5:det	_
4	"	"	PUNCT	fe	PunctType=Quot	5	punct	5:punct	SpaceAfter=No
5	perestroika	perestroika	NOUN	ncfs000	Gender=Fem|Number=Sing	1	nmod	1:nmod	SpaceAfter=No|ClusterId=3LB-CAST-111_C-4-s1.sn.9|ClusterType=Spec.other|MentionSpan=3-13
6	"	"	PUNCT	fe	PunctType=Quot	5	punct	5:punct	_
7	de	de	ADP	sps00	_	8	case	8:case	_
8	Mijail	Mijail	PROPN	np0000p	_	5	nmod	5:nmod	MWE=Mijail_Gorbachov|MWEPOS=PROPN|ClusterId=3LB-CAST-111_C-4-s1.sn.19|ClusterType=Spec.pe

In [ ]:
def read_infile(infile):
    answer, sent, labels = [], [], []
    with open(infile, "r", encoding="utf8") as fin:
        for line in fin:
            line = line.strip()
            if line == "":
                if len(sent) > 0:
                    answer.append({"words": sent, "labels": labels})
                sent, labels = [], []
                continue
            elif line[:1] == "#":
                continue
            splitted = line.split("\t")
            if not splitted[0].isdigit():
                continue
            tag = splitted[3] if splitted[5] == "_" else f"{splitted[3]},{splitted[5]}"
            sent.append(splitted[1])
            labels.append(tag)
    if len(sent) > 0:
        answer.append({"words": sent, "labels": labels})
    return answer

In [ ]:
train_data = read_infile("/content/es_ancora-ud-train.conllu")
dev_data = read_infile("/content/es_ancora-ud-dev.conllu")
for word, tag in zip(train_data[2]["words"], train_data[2]["labels"]):
    print(word, "\t"*4, tag)

Los 				 DET,Definite=Def|Gender=Masc|Number=Plur|PronType=Art
activos 				 NOUN,Gender=Masc|Number=Plur
en 				 ADP
divisas 				 NOUN,Gender=Fem|Number=Plur
en 				 ADP
poder 				 NOUN,Gender=Masc|Number=Sing
de 				 ADP
el 				 DET,Definite=Def|Gender=Masc|Number=Sing|PronType=Art
Banco 				 PROPN
Central 				 PROPN
y 				 CCONJ
el 				 DET,Definite=Def|Gender=Masc|Number=Sing|PronType=Art
Ministerio 				 PROPN
de 				 ADP
Finanzas 				 PROPN
se 				 PRON,Case=Acc|Person=3|PrepCase=Npr|PronType=Prs|Reflex=Yes
calculan 				 VERB,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin
en 				 ADP
dólares 				 NOUN,NumForm=Digit
estadounidenses 				 ADJ,Number=Plur
y 				 CCONJ
su 				 DET,Number=Sing|Person=3|Poss=Yes|PronType=Prs
valor 				 NOUN,Gender=Masc|Number=Sing
depende 				 VERB,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
de 				 ADP
el 				 DET,Definite=Def|Gender=Masc|Number=Sing|PronType=Art
cambio 				 NOUN,Gender=Masc|Number=Sing
oficial 				 ADJ,Number=Sing
rublo

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
tokens = tokenizer(train_data[2]["words"], is_split_into_words=True)
print(tokens.keys())
print(tokens["input_ids"])
print(tokenizer.convert_ids_to_tokens(tokens["input_ids"]))
print(tokens.word_ids())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[4, 1412, 5840, 1036, 16869, 1036, 2313, 1008, 1040, 5184, 4774, 1042, 1040, 5026, 1008, 16213, 1062, 17076, 30935, 1036, 3066, 9450, 1042, 1079, 2860, 7595, 1008, 1040, 2957, 3844, 2603, 1307, 30933, 1149, 14089, 1038, 4205, 1040, 5184, 4774, 1009, 5]
['[CLS]', 'Los', 'activos', 'en', 'divisas', 'en', 'poder', 'de', 'el', 'Banco', 'Central', 'y', 'el', 'Ministerio', 'de', 'Finanzas', 'se', 'calcula', '##n', 'en', 'dólares', 'estadounidenses', 'y', 'su', 'valor', 'depende', 'de', 'el', 'cambio', 'oficial', 'ru', '##bl', '##o', '-', 'dólar', 'que', 'establece', 'el', 'Banco', 'Central', '.', '[SEP]']
[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 28, 28, 28, 29, 30, 31, 32, 33, 34, None]


In [ ]:
word_ids = [0, 1, 1, 2, 3, 4, 5, 5, 5, 6]
mask = [first != second for first, second in zip(word_ids[:-1], word_ids[1:])]
print(mask)

[True, False, True, True, True, True, False, False, True]


In [ ]:
def make_last_subtoken_mask(mask):
    mask = mask[1:-1]
    is_last_word = [False] + list((first != second) for first, second in zip(mask[:-1], mask[1:])) + [True, False]
    return is_last_word

print(make_last_subtoken_mask(tokens.word_ids()))

[False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, True, True, True, True, True, True, True, False]


In [ ]:
from collections import Counter
import numpy as np
from torch.utils.data.dataset import Dataset

class UDDataset(Dataset):

    def __init__(self, data, tokenizer, min_count=3, tags=None):
        self.data = data
        self.tokenizer = tokenizer
        if tags is None:
            tag_counts = Counter([tag for elem in data for tag in elem["labels"]])
            self.tags_ = ["<PAD>", "<UNK>"] + [x for x, count in tag_counts.items() if count >= min_count]
        else:
            self.tags_ = tags
        self.tag_indexes_ = {tag: i for i, tag in enumerate(self.tags_)}
        self.unk_index = 1
        self.ignore_index = -100

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        tokenization = self.tokenizer(item["words"], is_split_into_words=True)
        last_subtoken_mask = make_last_subtoken_mask(tokenization.word_ids())
        answer = {"input_ids": tokenization["input_ids"], "mask": last_subtoken_mask}
        if "labels" in item:
            labels = [self.tag_indexes_.get(tag, self.unk_index) for tag in item["labels"]]
            zero_labels = np.array([self.ignore_index] * len(tokenization["input_ids"]), dtype=int)
            zero_labels[last_subtoken_mask] = labels
            answer["y"] = zero_labels
        return answer

In [ ]:
train_dataset = UDDataset(train_data, tokenizer)
for key, value in train_data[2].items():
    print(key, value)
for key, value in train_dataset[2].items():
    print(key, value)

words ['Los', 'activos', 'en', 'divisas', 'en', 'poder', 'de', 'el', 'Banco', 'Central', 'y', 'el', 'Ministerio', 'de', 'Finanzas', 'se', 'calculan', 'en', 'dólares', 'estadounidenses', 'y', 'su', 'valor', 'depende', 'de', 'el', 'cambio', 'oficial', 'rublo-dólar', 'que', 'establece', 'el', 'Banco', 'Central', '.']
labels ['DET,Definite=Def|Gender=Masc|Number=Plur|PronType=Art', 'NOUN,Gender=Masc|Number=Plur', 'ADP', 'NOUN,Gender=Fem|Number=Plur', 'ADP', 'NOUN,Gender=Masc|Number=Sing', 'ADP', 'DET,Definite=Def|Gender=Masc|Number=Sing|PronType=Art', 'PROPN', 'PROPN', 'CCONJ', 'DET,Definite=Def|Gender=Masc|Number=Sing|PronType=Art', 'PROPN', 'ADP', 'PROPN', 'PRON,Case=Acc|Person=3|PrepCase=Npr|PronType=Prs|Reflex=Yes', 'VERB,Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin', 'ADP', 'NOUN,NumForm=Digit', 'ADJ,Number=Plur', 'CCONJ', 'DET,Number=Sing|Person=3|Poss=Yes|PronType=Prs', 'NOUN,Gender=Masc|Number=Sing', 'VERB,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin', 'ADP', 'DET

In [ ]:
train_dataloader = iter(FieldBatchDataLoader(train_dataset, batch_size=8, device="cuda"))
for i in range(10):
    batch = next(train_dataloader)
    for field, data in batch.items():
        print(f"{field}:{np.shape(data)}", end="\t")
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


input_ids:torch.Size([8, 13])	mask:torch.Size([8, 13])	y:torch.Size([8, 13])	indexes:(8,)	
input_ids:torch.Size([8, 57])	mask:torch.Size([8, 57])	y:torch.Size([8, 57])	indexes:(8,)	
input_ids:torch.Size([8, 42])	mask:torch.Size([8, 42])	y:torch.Size([8, 42])	indexes:(8,)	
input_ids:torch.Size([8, 39])	mask:torch.Size([8, 39])	y:torch.Size([8, 39])	indexes:(8,)	
input_ids:torch.Size([8, 51])	mask:torch.Size([8, 51])	y:torch.Size([8, 51])	indexes:(8,)	
input_ids:torch.Size([8, 25])	mask:torch.Size([8, 25])	y:torch.Size([8, 25])	indexes:(8,)	
input_ids:torch.Size([8, 29])	mask:torch.Size([8, 29])	y:torch.Size([8, 29])	indexes:(8,)	
input_ids:torch.Size([8, 7])	mask:torch.Size([8, 7])	y:torch.Size([8, 7])	indexes:(8,)	
input_ids:torch.Size([8, 36])	mask:torch.Size([8, 36])	y:torch.Size([8, 36])	indexes:(8,)	
input_ids:torch.Size([8, 11])	mask:torch.Size([8, 11])	y:torch.Size([8, 11])	indexes:(8,)	


## Модель

In [ ]:
import torch.nn as nn
from transformers.optimization import AdamW

class BasicTransformersTaggingModel(nn.Module):

    def __init__(self, model, labels_number, lr=1e-5, device="cpu", **kwargs):
        super(BasicTransformersTaggingModel, self).__init__()
        self.model = model
        self.labels_number = labels_number
        self.build_network(labels_number)
        # определяем функцию потерь
        self.log_softmax = nn.LogSoftmax(dim=-1)
        self.criterion = nn.NLLLoss(reduction="mean", ignore_index=-100)
        self.device = device
        if self.device is not None:
            self.to(self.device)
        self.optimizer = AdamW(self.parameters(), lr=lr, weight_decay=0.01)

    @property
    def hidden_size(self):
        return self.model.config.hidden_size

    def forward(self, input_ids, **kwargs):
        raise NotImplementedError("You should implement forward pass in your derived class.") 

    def train_on_batch(self, x, y, mask=None):
        self.train()
        self.optimizer.zero_grad()
        loss = self._validate(x, y, mask=mask)
        loss["loss"].backward()
        self.optimizer.step()
        return loss

    def validate_on_batch(self, x, y, mask=None):
        self.eval()
        with torch.no_grad():
            return self._validate(x, y, mask=mask)

    def _validate(self, x, y, mask=None):
        if self.device is not None:
            y = y.to(self.device)
        log_probs = self(**x) #   self.forward(x) = self.__call__(x)
        # log_probs.shape = (B, L, K)
        # для NLLLoss нужно (B, K, L)
        loss = self.criterion(log_probs.permute(0, 2, 1), y)
        _, labels = torch.max(log_probs, dim=-1)
        # print("y_corr", y[0].cpu().numpy())
        # print("y_pred", labels[0].cpu().numpy())
        # print("mask", mask[0].cpu().numpy())
        batch_labels = [None] * len(labels)
        batch_probs = [None] * len(labels)
        for i, elem in enumerate(labels):
            if mask is None:
                curr_mask = [True] * len(elem)
            else:
                curr_mask = mask[i].bool()
            batch_labels[i] = elem[curr_mask]
            batch_probs[i] = log_probs[i][curr_mask]
        return {"loss": loss, "labels": batch_labels, "probs": batch_probs}

class TransformersTaggingModel(BasicTransformersTaggingModel):

    def build_network(self, labels_number):
        self.proj_layer = torch.nn.Linear(self.hidden_size, self.labels_number)
        return self

    def forward(self, input_ids, **kwargs):
        input_ids = input_ids.to(self.device)
        cls_output = self.model(input_ids)["last_hidden_state"]
        logits = self.proj_layer(cls_output)
        log_probs = self.log_softmax(logits)
        return log_probs

In [ ]:
from transformers import BertModel
from sklearn.metrics import accuracy_score

bert_model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

model = TransformersTaggingModel(bert_model, labels_number=len(train_dataset.tags_), device="cuda")
train_dataloader = iter(FieldBatchDataLoader(train_dataset, batch_size=8, device="cuda"))
batch = next(train_dataloader)
labels = batch["y"]
for i in range(100):
    batch_output = model.train_on_batch(batch, labels, batch["mask"])
    loss = batch_output["loss"].item()
    # посчитаем потокенную точность
    ## извлекаем ответы
    pred_labels_flat = [x for sent in batch_output["labels"] for x in sent.cpu().numpy()]
    corr_labels_flat = [x for sent in labels for x in sent.cpu().numpy() if x not in [-100, 0]]
    # print(pred_labels_flat)
    # print(corr_labels_flat)
    if i < 5 or (i+1) % 10 == 0:
        print(i, f"{loss:.4f} {100*accuracy_score(corr_labels_flat, pred_labels_flat):.2f}")
        # print(batch_output["labels"][0].cpu().numpy())
        # print(labels[0].cpu().numpy())
        # print(batch_output["labels"][1].cpu().numpy())
        # print(labels[1].cpu().numpy())
        # print(batch["mask"][0].cpu().numpy())
print(model.validate_on_batch(batch, labels, batch["mask"])["loss"].item())

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

0 5.8155 0.00
1 5.6994 0.00
2 5.6376 0.00
3 5.5600 0.00
4 5.4950 0.00
9 5.0813 25.32
19 4.1709 59.49
29 3.2547 79.75
39 2.4823 87.34
49 1.8858 94.94
59 1.3923 100.00
69 0.9959 100.00
79 0.7342 100.00
89 0.5475 100.00
99 0.4298 100.00
0.16584831476211548


In [ ]:
def get_batch_metrics(pred_labels, labels):
    correct, total, seq_correct = 0, 0, 0
    for curr_pred_labels, curr_labels in zip(pred_labels, labels):
        curr_labels = [label for label in curr_labels if label not in [0, -100]]
        assert len(curr_pred_labels) == len(curr_labels), f"{len(curr_pred_labels)}-{len(curr_labels)}"
        are_equal = [int(x==y) for x, y in zip(curr_pred_labels, curr_labels)]
        correct += sum(are_equal)
        total += len(are_equal)
        seq_correct += int(all(are_equal))
    return correct, total, seq_correct, len(labels)

def update_metrics(metrics, batch_output, batch_labels):
    n_batches = metrics["n_batches"]
    metrics["loss"] = (metrics["loss"] * n_batches + batch_output["loss"].item()) / (n_batches + 1)
    metrics["n_batches"] += 1
    correct, total, seq_correct, seq_total = get_batch_metrics(batch_output["labels"], batch_labels.cpu().tolist())
    metrics["correct"] += correct
    metrics["total"] += total
    metrics["sent_correct"] += seq_correct
    metrics["sent_total"] += seq_total
    metrics["accuracy"] = metrics["correct"] / max(metrics["total"], 1)
    metrics["sent_accuracy"] = metrics["sent_correct"] / max(metrics["sent_total"], 1)

In [ ]:
import tqdm

def do_epoch(model, dataloader, mode="validate", epoch=1):
    metrics = {"correct": 0, "total": 0, "loss": 0.0, "n_batches": 0, "sent_correct": 0, "sent_total": 0}
    func = model.train_on_batch if mode == "train" else model.validate_on_batch
    progress_bar = tqdm.notebook.tqdm(dataloader, leave=True)
    progress_bar.set_description(f"{mode}, epoch={epoch}")
    for batch in progress_bar:
        batch_answers, mask = batch["y"], batch.get("mask")
        if mask is not None:
            mask = mask.bool()
        batch_output = func(batch, batch_answers, mask=mask)
        update_metrics(metrics, batch_output, batch_answers)
        postfix = {"loss": round(metrics["loss"], 4), 
                   "acc": round(100 * metrics["accuracy"], 2),
                   "sent_acc": round(100 * metrics["sent_accuracy"], 2)}
        progress_bar.set_postfix(postfix)
    return metrics

In [ ]:
train_dataset = UDDataset(train_data, tokenizer)
dev_dataset = UDDataset(dev_data, tokenizer, tags=train_dataset.tags_)
train_dataloader = iter(FieldBatchDataLoader(train_dataset, batch_size=8, device="cuda"))
dev_dataloader = iter(FieldBatchDataLoader(dev_dataset, batch_size=8, device="cuda"))

bert_model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = TransformersTaggingModel(bert_model, labels_number=len(train_dataset.tags_), device="cuda")
best_val_acc = 0.0
checkpoint = "checkpoint_best.pt"
for epoch in range(3):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)
        # print("Saving ")
model.load_state_dict(torch.load(checkpoint))
do_epoch(model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel 

  0%|          | 0/1786 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/1786 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/1786 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

{'accuracy': 0.9809505988023952,
 'correct': 52422,
 'loss': 0.09237772727538131,
 'n_batches': 207,
 'sent_accuracy': 0.6064087061668681,
 'sent_correct': 1003,
 'sent_total': 1654,
 'total': 53440}

In [ ]:
def predict_with_model(model, dataset):
    model.eval()
    dataloader = FieldBatchDataLoader(dataset, device=model.device)
    answer = [None] * len(dataset)
    for batch in dataloader:
        with torch.no_grad():
            batch_answer = model(**batch)
        _, labels = torch.max(batch_answer, dim=-1)
        for i, mask, sent_labels in zip(batch["indexes"], batch["mask"], labels):
            # mask = dataset[i]["mask"]
            mask = mask.bool().cpu().numpy()
            answer[i] = np.take(dataset.tags_, sent_labels.cpu().numpy()[mask])
    return answer

In [ ]:
test_data = read_infile("/content/es_ancora-ud-test.conllu")
test_dataset = UDDataset(test_data, tokenizer, tags=train_dataset.tags_) 
pred_labels = predict_with_model(model, test_dataset)
corr_labels = [elem["labels"] for elem in test_data]
correct, total, sent_correct, sent_total = get_batch_metrics(pred_labels, corr_labels)
print(f"Точность: {(100*correct/total):.2f} ({correct}/{total})")
print(f"Точность по предложениям: {(100*sent_correct/sent_total):.2f} ({sent_correct}/{sent_total})")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Точность: 98.07 (52567/53604)
Точность по предложениям: 61.59 (1060/1721)
